In [1]:
from pythreejs import *
from IPython.display import display
from math import pi
import numpy as np

This test examines the inner workings of the Geometry classes
Geometries should be creatable via the helper extras/Geometries classes
as well as procedurally creating the vertices yourself

In [2]:
g = BoxGeometry(
    width=5, 
    height=10, 
    depth=15,
    widthSegments=5, 
    heightSegments=10,
    depthSegments=15)

In [3]:
g

Preview(child=BoxGeometry(depth=15.0, depthSegments=15, height=10.0, heightSegments=10, width=5.0, widthSegmen…

In [4]:
g2 = Geometry.from_geometry(g)

In [5]:
np.shape(g2.vertices), len(g2.faces)

((0,), 0)

In [6]:
g2

Preview(child=Geometry(colors=['#ffffff']), shadowMap=WebGLShadowMap())

In [7]:
g3 = Geometry(vertices=g2.vertices[:], faces=g2.faces[:])

In [8]:
g3

Preview(child=Geometry(colors=['#ffffff']), shadowMap=WebGLShadowMap())

In [9]:
g2.faces

()

In [10]:

vertices = []
vertices.append([-50, 50, 0])
vertices.append([50, -50, 0])
vertices.append([50, 50, 0])
vertices.append([50, 50, 50])
normal = Tuple((0,0,1))
vertexcolors = ['#000000', '#0000ff', '#00ff00', '#ff0000',
     '#00ffff', '#ff00ff', '#ffff00', '#ffffff']
face = Tuple((0, 1, 2, normal, vertexcolors))
faces = [[0, 1, 2], [1, 2, 3], [3, 0, 1]]
faces = [f + [None, [vertexcolors[i] for i in f], None] for f in faces]
g3 = Geometry(vertices=vertices, faces=faces)
g3

Preview(child=Geometry(colors=['#ffffff'], faces=((0, 1, 2, None, ('#000000', '#0000ff', '#00ff00'), None), (1…

## Importing FreeCAD

In [11]:
import sys, os

JUPYTER_REPO_PATH = "/opt/jupyter_freecad/"

sys.path.append("/opt/freecad/freecad_build/lib")
sys.path.append(JUPYTER_REPO_PATH + "Jupyter")

import FreeCAD, FreeCADGui
FreeCADGui.setupWithoutGUI()

Creating a document with objects and a scene graph to be iterated over later on.

In [12]:
from pivy import *
from pivy.gui import soqt

VRML_PATH = JUPYTER_REPO_PATH + "three.js-r117/examples/models/vrml/freecad.wrl"

doc = FreeCAD.newDocument()
doc.addObject("Part::Box","Box")
doc.addObject("Part::Cylinder","Cylinder")
doc.addObject("Part::Sphere","Sphere")
doc.addObject("Part::Torus","Torus")
doc.recompute()

from pivy import coin
root = coin.SoSeparator()
for obj in doc.Objects:
    root.addChild(FreeCADGui.subgraphFromObject(obj))

Now we need to go over the scene graph and extract the edges and surfaces. First we grab the wire representation (chosing one of the switch node children. The switch node allows switching between different FreeCAD views such as Mesh, Surface etc.)

To view the scene graph in a convenient way open `FreeCAD > Tools > view scene graph`. That's how I selected the following `SoIndexedLineSet` and the corresponding `SoCoordinate3` object. The coordinates all switch children refer to are always the coordinates in the root of the object. So one level below the document root node.

In [13]:
def so_col_to_hex(so_color):
    color = (int(so_color[0]*255), 
                  int(so_color[1]*255),
                  int(so_color[2]*255))
    hex_col = "#{0:02x}{1:02x}{2:02x}".format(color[0],
                                              color[1],
                                              color[2])
    return hex_col

def extract_so_indices(so_node):
    """
    Returns list of indices from pivy.coin
    scene objects 'SoIndexedLine' and 'SoIndexedFace'
    """
    faces = list(so_node.coordIndex)
    indices = []
    curr_line = []
    for i in faces:
        if i == -1:
            indices.append(curr_line)
            curr_line = []
            continue
        curr_line.append(i)
    return indices

def generate_line_vertices(line_indices, coord_vals):
    line_vertices = []
    for i in line_indices:
        line_vertices.append(coord_vals[i[0]])
        line_vertices.append(coord_vals[i[1]])
    return line_vertices

def extract_faces(obj_node):
    # The names in the following comments refer to the
    # "Name" field in FreeCAS > tools > Scene Inspector
    # The types refer to the field "Inventor Tree".
    so_coord = obj_node[1]
    so_wireframe = obj_node[2][2] # Name: Wireframe
    so_shaded = obj_node[2][1] # Name: Shaded
    so_faces = so_shaded[6] # Type: SoBrepFaceSet
    so_wireframe_edges = so_wireframe[0][3] # Type: SoBrepEdgeSet
    so_shaded_material = so_shaded[2]
    so_wireframe_material = so_wireframe[0][1]
    coords = list(so_coord.point)

    #print(lines)
    #print(coords)
    #print(so_coord)
    #print(so_lines)
    so_shaded_color = so_shaded_material.ambientColor.getValues()[0]
    so_shaded_emissive_color = so_shaded_material.emissiveColor.getValues()[0]
    so_wireframe_color = so_wireframe_material.ambientColor.getValues()[0]
    face_color = (so_shaded_color[0], so_shaded_color[1], so_shaded_color[2])
    face_emissive_color = (so_shaded_color[0], so_shaded_color[1], so_shaded_color[2])
    line_color = (so_wireframe_color[0], so_wireframe_color[1], so_wireframe_color[2])
    face_color = so_col_to_hex(face_color)
    face_emissive_color = so_col_to_hex(face_emissive_color)
    line_color = so_col_to_hex(line_color)
    face_transparency = so_shaded_material.transparency[0]

    """
    print(face_emissive_color)
    print(line_color)
    print(so_wireframe_edges)
    print(face_color)
    print(so_shaded_color[3] == face_transparency)
    print(list(so_shaded_material.shininess))
    print(list(so_shaded_material.emissiveColor))
    print(face_transparency)
    print(list(so_shaded_material.specularColor))
    print(list(so_shaded_material.diffuseColor))
    """


    coord_vals = [list(x) for x in coords]
    
    face_indices = extract_so_indices(so_faces)
    line_indices = extract_so_indices(so_wireframe_edges)
    line_vertices = generate_line_vertices(line_indices, coord_vals)

    #print(face_indices)
    #print(coord_vals)
    return coord_vals, face_indices, line_vertices, face_color, line_color, face_transparency

def create_geometry(obj_node):
    coord_vals, face_indices, line_vertices, face_color, line_color, face_transparency = extract_faces(obj_node)
    
    vertices = np.asarray(coord_vals, dtype='float32')
    faces = np.asarray(face_indices, dtype='uint16').ravel()  # We need to flatten index array
    vertexcolors = np.asarray([(1,0,0)]*len(coord_vals), dtype='float32')

    faceGeometry = BufferGeometry(attributes=dict(
        position=BufferAttribute(vertices, normalized=False),
        index=BufferAttribute(faces, normalized=False),
        colors=BufferAttribute(vertexcolors)
    ))
    
    faceGeometry.exec_three_obj_method('computeFaceNormals')


    object_mesh = Mesh(
        geometry=faceGeometry,
        material=MeshLambertMaterial(color=face_color, opacity=face_transparency),
        position=[0,0,0]   # Center the cube
    )
    
    linesgeom = Geometry(vertices=line_vertices)
    lines = Line(geometry=linesgeom, 
             material=LineBasicMaterial(linewidth=5, color=line_color), 
             type='LinePieces',
            )
    return [object_mesh, lines]

def render_objects(root_node):
    geometries = []
    for node in root_node:
        if type(node) is coin.SoSeparator:
            res = create_geometry(node)
            geometries.extend(res)
    camera = PerspectiveCamera(
        position=[40, 40, 40], fov=40,
        children=[DirectionalLight(color='#ffffff', position=[20, 20, 10], intensity=1)])
    children = [camera, AmbientLight(color='#ffffff')]
    children.extend(geometries)
    scene = Scene(children=children)
    controls = [OrbitControls(controlling=camera)]

    renderer = Renderer(camera=camera, background='black', background_opacity=1,
                        scene=scene, controls=controls,
                        width=900, height=900)
    return renderer

render_objects(root)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(20.0, 20.0, 10.0), quaternion=(0.0, 0.0…